In [10]:
import cv2
import numpy as np
import os
import pandas as pd
import csv

from sklearn.cluster import MiniBatchKMeans
# from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import glob
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import metrics
from commonFunctions import *

In [11]:
def shadow_removalDialate(img):
    # img2 = cv2.imread(f'../Dataset_0-5/men/{num}/4_men (49).JPG')
    img2 = cv2.resize(img, (256, 256))
    
    img_hls = cv2.cvtColor(img2, cv2.COLOR_BGR2HLS)

    # define lower and upper bounds for blue color in HLS format
    lower_blue = np.array([0, 0, 60])
    upper_blue = np.array([20, 255, 255])

    # create a mask for blue color in HLS format
    mask = cv2.inRange(img_hls, lower_blue, upper_blue)
    # daialation
    kernel = np.ones((3,3),np.uint8)
    mask = cv2.dilate(mask,kernel,iterations = 8)
    # apply the mask to the original image
    result = cv2.bitwise_and(img2, img2, mask=mask)
    return result

In [12]:
def preprocessing(img,img2):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply histogram equalization to enhance contrast and remove lighting effects
    equalized = cv2.equalizeHist(gray)
    # # Find the contours of the binary image
    contours, hierarchy = cv2.findContours(equalized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_contour = max(contours, key=cv2.contourArea)

    # # Draw the largest contour on the original image
    cv2.drawContours(img2, [max_contour], 0, (0, 255, 0), 2)
    # # Show the image with the largest contour drawn
    # # Create a bounding box around the hand
    x, y, w, h = cv2.boundingRect(max_contour)

    # # Crop the image to the bounding box around the hand
    hand = equalized[y:y+h, x:x+w]
    img3 = cv2.resize(hand, (256, 256))

    return img3

In [13]:
im= cv2.imread("../Dataset_0-5/men/0/0_men (2).jpg")
hand = shadow_removalDialate(im)
result = preprocessing(hand,im)
        
akaze = cv2.AKAZE_create()
kp, des = akaze.detectAndCompute(result, None)
print(np.shape(des))

(78, 61)


In [14]:
arr = ["0","1","2","3","4","5"]
dico = []


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            hand = shadow_removalDialate(im)
            result = preprocessing(hand,im)
                            
            akaze = cv2.AKAZE_create()
            kp, des = akaze.detectAndCompute(result, None)
            for d in des:
                dico.append(d)
        except:
            pass
        

In [ ]:
arr = ["0","1","2","3","4","5"]


for label in arr:
    dirList = glob.glob("../Dataset_0-5/Women/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            hand = shadow_removalDialate(im)
            result = preprocessing(hand,im)
                            
            akaze = cv2.AKAZE_create()
            kp, des = akaze.detectAndCompute(result, None)
            for d in des:
                dico.append(d)
        except:
            pass
        

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=60)
kmeans.fit(dico)
joblib.dump(kmeans, "model/akazakmeans.pkl")

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
histo_list = []
Y = []
arr = ["0","1","2","3","4","5"]

lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            hand = shadow_removalDialate(im)
            result = preprocessing(hand,im)
                            
            akaze = cv2.AKAZE_create()
            kp, des = akaze.detectAndCompute(result, None)
            histo = np.zeros(60)
            nkp = np.size(kp)
            for d in des:
                idx = kmeans.predict([d])
                histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

            histo_list.append(histo)
            Y.append(int(label))
        except:
            pass

In [ ]:
arr = ["0","1","2","3","4","5"]



for label in arr:
    dirList = glob.glob("../Dataset_0-5/Women/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            hand = shadow_removalDialate(im)
            result = preprocessing(hand,im)
                            
            akaze = cv2.AKAZE_create()
            kp, des = akaze.detectAndCompute(result, None)
            histo = np.zeros(60)
            nkp = np.size(kp)
            for d in des:
                idx = kmeans.predict([d])
                histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

            histo_list.append(histo)
            Y.append(int(label))
        except:
            pass

In [ ]:
X = np.array(histo_list)
train_x,test_x,train_y,test_y = train_test_split(histo_list,Y, test_size = 0.3)
classifier=SVC(kernel="poly", random_state=6, degree=6)
classifier.fit(train_x,train_y)
joblib.dump(classifier, "model/surf_model.pkl")

['model/surf_model.pkl']

In [ ]:
from sklearn import metrics

prediction=classifier.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.5063694267515924
Precision=  0.5151944337623396
Recall=  0.5063694267515924
F1 Score=  0.50905710196612
Confusion Matrix=  [[32  6  7  5  1  0]
 [ 5 36  9  2  2  3]
 [11  7 17  6  3  0]
 [ 1  7  9 22 13  2]
 [ 4  2  4 12 13  9]
 [ 1  3  5  5 11 39]]
Classification Report=                precision    recall  f1-score   support

           0       0.59      0.63      0.61        51
           1       0.59      0.63      0.61        57
           2       0.33      0.39      0.36        44
           3       0.42      0.41      0.42        54
           4       0.30      0.30      0.30        44
           5       0.74      0.61      0.67        64

    accuracy                           0.51       314
   macro avg       0.50      0.49      0.49       314
weighted avg       0.52      0.51      0.51       314



In [16]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier and fit it to the training data
rfc = RandomForestClassifier(n_estimators=300)
rfc.fit(train_x, train_y)

RandomForestClassifier(n_estimators=300)

In [17]:
prediction=rfc.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.43312101910828027
Precision=  0.4404583113297712
Recall=  0.43312101910828027
F1 Score=  0.4339522932172457
Confusion Matrix=  [[34  0  7  5  3  3]
 [ 8 35  4  5  2  0]
 [ 5 14 13 10  3  3]
 [ 2  5 14  9  7  3]
 [ 2  6  7 18 10 10]
 [ 4  2  3  5 18 35]]
Classification Report=                precision    recall  f1-score   support

           0       0.62      0.65      0.64        52
           1       0.56      0.65      0.60        54
           2       0.27      0.27      0.27        48
           3       0.17      0.23      0.20        40
           4       0.23      0.19      0.21        53
           5       0.65      0.52      0.58        67

    accuracy                           0.43       314
   macro avg       0.42      0.42      0.42       314
weighted avg       0.44      0.43      0.43       314

